## Demo

In [1]:
import pandas as pd
from core.retriever import Retriever

r = Retriever(
    csv_path="data/data.csv",
    model_name="Qwen/Qwen3-Embedding-0.6B",  retrieve_k=64,
    reranker_name="BAAI/bge-reranker-v2-m3", top_k=8,  # optional
    rebuild_index=False
)

r.load()

query = "hicup"
nodes = r.search(query)

pd.DataFrame(
    [{"id": n.metadata["id"], "text": n.text, "score": n.score} for n in nodes]
).head(3)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


,id,text,score
0,3d6ab356-e155-4ad5-9a89-219b1501c39c,อาการสะอึกซ้ำซาก (Recurrent hiccups) คืออาการส...,-1.267578
1,7ec19a40-fe3a-4f1d-9167-99f26b76b8d3,วัคซีนฮิบคือวัคซีนที่ป้องกันการติดเชื้อแบคทีเร...,-2.349609
2,4a8f6437-6f3c-4869-b59d-d75c83397980,วัคซีนฮิบใช้ป้องกันโรคเยื่อหุ้มสมองอักเสบ ปอดบ...,-3.441406


## Implementation

### Retrival

In [2]:
import pandas as pd
from llama_index.core.schema import TextNode
from llama_index.vector_stores.lancedb import LanceDBVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, VectorStoreIndex, StorageContext


df = pd.read_csv("data/demo.csv")

nodes = [
    TextNode(text=row["text"],metadata={"id": row["id"]})
        for _, row in df.iterrows() if not pd.isna(row['text'])
]

## Embedding
Settings.embed_model = HuggingFaceEmbedding(model_name="Qwen/Qwen3-Embedding-0.6B", device="cuda", normalize=True,)

## Vector DB
vector_store = LanceDBVectorStore(uri="./.cache/lancedb", table_name="embeddings")
storage = StorageContext.from_defaults(vector_store=vector_store)

## Retriever
index = VectorStoreIndex(nodes, storage_context=storage, show_progress=True)
index.storage_context.persist(persist_dir='.cache')
retriever = index.as_retriever(similarity_top_k=64)


## Inference
query = "hicup"
# Retrieve
nodes = retriever.retrieve(query)

pd.DataFrame([{"id":o.metadata['id'], "text": o.text, "score": o.score} for o in nodes])

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

,id,text,score
0,48371e11-6211-4918-bc8f-07c3ea4e9d23,โรคแพนิคหรือโรคตื่นตระหนกเป็นภาวะที่มีอาการวิต...,0.198952
1,625e5dc6-4491-4a54-87dd-04b819538a26,อาการของโรคแพนิคแบ่งเป็นสองกลุ่มคือ อาการทางร่...,0.177184
2,accad5f2-eced-4ede-9f42-7fe80bd1da80,สาเหตุหลักของโรคแพนิคคือการทำงานผิดปกติของสมอง...,0.174470


### Reranker

In [3]:
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

## Reranker
reranker = FlagEmbeddingReranker(model="BAAI/bge-reranker-v2-m3", top_n=3, use_fp16=True)

# Ranking
out = reranker.postprocess_nodes(nodes, query_str=query)
pd.DataFrame([{"id":o.metadata['id'], "text": o.text, "score": o.score} for o in out])

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


,id,text,score
0,48371e11-6211-4918-bc8f-07c3ea4e9d23,โรคแพนิคหรือโรคตื่นตระหนกเป็นภาวะที่มีอาการวิต...,-9.351562
1,accad5f2-eced-4ede-9f42-7fe80bd1da80,สาเหตุหลักของโรคแพนิคคือการทำงานผิดปกติของสมอง...,-10.148438
2,625e5dc6-4491-4a54-87dd-04b819538a26,อาการของโรคแพนิคแบ่งเป็นสองกลุ่มคือ อาการทางร่...,-10.687500
